SQL adalah salah satu bahasa populer untuk pemrosesan dan analisis data. Spark mendukung SQL untuk memproses DataFrame.

Kita akan menggunakan data yang sama dengan yg digunakan pada bab eksplorasi DataFrame.


In [ ]:
#%pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Inisialisasi spark session untuk berinteraksi dengan Spark cluster

In [3]:
spark = SparkSession.builder.appName('DataFrame Basics').getOrCreate()

23/10/07 00:38:00 WARN Utils: Your hostname, dl247-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.15.135 instead (on interface ens33)
23/10/07 00:38:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/07 00:38:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Download dataset

In [4]:
#!wget https://raw.githubusercontent.com/urfie/SparkSQL-dengan-Hive/main/datasets/indonesia2013-2015.csv
!wget https://github.com/urfie/SparkSQL-dengan-Hive/raw/main/datasets/application_record_header.csv.gz

--2023-10-07 00:38:13--  https://github.com/urfie/SparkSQL-dengan-Hive/raw/main/datasets/application_record_header.csv.gz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/urfie/SparkSQL-dengan-Hive/main/datasets/application_record_header.csv.gz [following]
--2023-10-07 00:38:14--  https://raw.githubusercontent.com/urfie/SparkSQL-dengan-Hive/main/datasets/application_record_header.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3175443 (3,0M) [application/octet-stream]
Saving to: ‘application_record_header.csv.gz’

application_record_ 100%[===================>]   3,03M  3,03MB/s    in 1

Load ke dataframe

In [5]:
df = spark.read.csv("application_record_header.csv.gz", header=True, inferSchema=True)

Sebelum menggunakan SQL, kita perlu membuat temporary table dari dataframe yang akan kita olah.

Gunakan fungsi `createOrReplaceTempView(nama_tabel)` pada dataframe tersebut.

In [6]:
df.createOrReplaceTempView("app_record")

Selanjutnya kita bisa menggunakan nama tabel yang sudah kita definisikan dalam SQL statement.

Untuk mengeksekusi SQL statement, kita gunakan fungsi `sql(sqlstatement)` pada spark session.

In [7]:
spark.sql("select count(*) from app_record").show()

+--------+
|count(1)|
+--------+
|  438557|
+--------+



In [8]:
spark.sql("select * from app_record limit 5").show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|         1

In [9]:
spark.sql("select distinct NAME_EDUCATION_TYPE from app_record").show(truncate = False)

+-----------------------------+
|NAME_EDUCATION_TYPE          |
+-----------------------------+
|Academic degree              |
|Incomplete higher            |
|Secondary / secondary special|
|Lower secondary              |
|Higher education             |
+-----------------------------+



In [10]:
mydata = (('Academic degree',3),
    ('Incomplete higher',4),
    ('Secondary / secondary special',2),
    ('Lower secondary',1),
    ('Higher education',5))

ref_edu = spark.createDataFrame(mydata).toDF("NAME_EDUCATION_TYPE", "EDU_LEVEL")
ref_edu.createOrReplaceTempView("ref_edu")
spark.sql("select * from ref_edu").show()

+--------------------+---------+
| NAME_EDUCATION_TYPE|EDU_LEVEL|
+--------------------+---------+
|     Academic degree|        3|
|   Incomplete higher|        4|
|Secondary / secon...|        2|
|     Lower secondary|        1|
|    Higher education|        5|
+--------------------+---------+



In [11]:
spark.sql("""SELECT edu_level, count(1) FROM
              (SELECT ref_edu.EDU_LEVEL as edu_level
                FROM app_record LEFT JOIN ref_edu
                ON app_record.NAME_EDUCATION_TYPE=ref_edu.NAME_EDUCATION_TYPE)
             GROUP BY edu_level SORT BY edu_level""").write.saveAsTable(name="aggregated_edu", mode="overwrite")

In [13]:
spark.sql("show tables").show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|  default|aggregated_edu|      false|
|         |    app_record|       true|
|         |       ref_edu|       true|
+---------+--------------+-----------+



In [15]:
spark.sql("describe formatted aggregated_edu").show(truncate=False)

+----------------------------+--------------------------------------------------------+-------+
|col_name                    |data_type                                               |comment|
+----------------------------+--------------------------------------------------------+-------+
|edu_level                   |bigint                                                  |null   |
|count(1)                    |bigint                                                  |null   |
|                            |                                                        |       |
|# Detailed Table Information|                                                        |       |
|Database                    |default                                                 |       |
|Table                       |aggregated_edu                                          |       |
|Created Time                |Sat Oct 07 00:39:23 WIB 2023                            |       |
|Last Access                 |UNKNOWN   